In [1]:
import cv2
import numpy as np
import time
def nwlbp_calculation(img):
    # Taking inner and outer layer neighbourhood pixels in different directions of NWALBP template window
    i0 = img[1:4, 2:5]   # 0 degree
    i45 = img[0:3, 2:5]  # 45 degree
    i90 = img[0:3, 1:4]  # 90 degree
    i135 = img[0:3, 0:3]  # 135 degree

    i180 = img[1:4, 0:3]  # 180 degree
    i225 = img[2:5, 0:3]  # 225 degree
    i270 = img[2:5, 1:4]  # 270 degree
    i315 = img[2:5, 2:5]  # 315 degree

    # Krish operator in eight directions
    k0 = np.array([[0, 2, 1], [0, 2, 1], [0, 2, 1]])  # 0 degree
    k45 = np.array([[0, 1, 1], [2, 2, 1], [0, 2, 0]])  # 45 degree
    k90 = np.array([[1, 1, 1], [2, 2, 2], [0, 0, 0]])  # 90 degree
    k135 = np.array([[1, 1, 0], [1, 2, 2], [0, 2, 0]])  # 135 degree
    k180 = np.array([[1, 2, 0], [1, 2, 0], [1, 2, 0]])  # 180 degree
    k225 = np.array([[0, 2, 0], [1, 2, 2], [1, 1, 0]])  # 225 degree
    k270 = np.array([[0, 0, 0], [2, 2, 2], [1, 1, 1]])  # 270 degree
    k315 = np.array([[0, 2, 0], [2, 2, 1], [0, 1, 1]])  # 315 degree

    w = np.zeros((3, 3), dtype=np.uint8)

    # Calculating corresponding weights
    w[1, 2] = np.sum(i0 * k0) * (1 / 9)  # w0
    w[0, 2] = np.sum(i45 * k45) * (1 / 9)  # w1
    w[0, 1] = np.sum(i90 * k90) * (1 / 9)  # w2
    w[0, 0] = np.sum(i135 * k135) * (1 / 9)  # w3
    w[1, 0] = np.sum(i180 * k180) * (1 / 9)  # w4
    w[2, 0] = np.sum(i225 * k225) * (1 / 9)  # w5
    w[2, 1] = np.sum(i270 * k270) * (1 / 9)  # w6
    w[2, 2] = np.sum(i315 * k315) * (1 / 9)  # w7
    w[1, 1] = img[2, 2]

    return w  # Returning a 3x3 window of weights calculated


def myfcn_LBP(g):
    sz = g.shape
    rows = sz[0]
    columns = sz[1]
    center = (rows // 2, columns // 2)
    result = np.zeros((rows, columns), dtype=int)

    for i in range(rows):
        for j in range(columns):
            if g[center] <= g[i, j]:
                result[i, j] = 1

    return result


def myfcn_binaryToDecimal(g):
    sum_val = 0
    k = 1
    sz = g.shape
    elements = np.zeros(8, dtype=int)

    for j in range(sz[1] - 1):
        i = 0
        elements[k - 1] = g[i, j]
        k += 1

    for i in range(sz[0] - 1):
        j = sz[1] - 1
        elements[k - 1] = g[i, j]
        k += 1

    for j in range(sz[1] - 2, 0, -1):
        i = sz[0] - 1
        elements[k - 1] = g[i, j]
        k += 1

    for i in range(sz[0] - 2, 0, -1):
        j = 0
        elements[k - 1] = g[i, j]
        k += 1

    szele = elements.size
    reversed_array = np.flip(elements)

    for j in range(szele):
        sum_val += reversed_array[j] * 2**(szele - j - 1)

    return sum_val

start_time = time.time()
# Load an example image (replace with your own image)
original_image = cv2.imread('lena_std.tif', cv2.IMREAD_GRAYSCALE)
original_image = cv2.resize(original_image, (511, 511))

# Specify the amount of padding on all sides
top_pad = 2
bottom_pad = 2
left_pad = 2
right_pad = 2

# Define the value for padding (e.g., 0 for black, 255 for white)
padding_value = 0

# Use the np.pad function to add padding
padded_image = np.pad(original_image, ((top_pad, bottom_pad), (left_pad, right_pad)), mode='constant', constant_values=padding_value)

# Display the padded image
#cv2.imshow('Padded Image', padded_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# NWALBP calculation
sz = padded_image.shape
nwalbp_image = np.zeros_like(original_image, dtype=int)

# Initialize a cell array to store the 3x3 windows
# Loop through the image and extract 3x3 windows
for i in range(2, sz[0] - 2):
    for j in range(2, sz[1] - 2):
        # Extract the 5 * 5 window centered at (i, j)
        window = padded_image[i-2:i+3, j-2:j+3]
        # Store the window in the cell array
        g = nwlbp_calculation(window)
        img_lbp = myfcn_LBP(g)
        img_decimal = myfcn_binaryToDecimal(img_lbp)
        nwalbp_image[i-2, j-2] = img_decimal

# Display the NWALBP image
cv2.imshow('NWALBP Image', nwalbp_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()
end_time = time.time()

time_taken_nwalbp = end_time - start_time

print(time_taken_nwalbp)

201.45676970481873


In [4]:
import cv2
import numpy as np
import time
def nwlbp_calculation(img):
    # Taking inner and outer layer neighborhood pixels in different directions of NWALBP template window
    i0 = img[1:4, 2:5]  # 0 degree
    i45 = img[0:3, 2:5]  # 45 degree
    i90 = img[0:3, 1:4]  # 90 degree
    i135 = img[0:3, 0:3]  # 135 degree
    i180 = img[1:4, 0:3]  # 180 degree
    i225 = img[2:5, 0:3]  # 225 degree
    i270 = img[2:5, 1:4]  # 270 degree
    i315 = img[2:5, 2:5]  # 315 degree

    # Krish operator in eight directions
    k0 = np.array([[0, 2, 1], [0, 2, 1], [0, 2, 1]])  # 0 degree
    k45 = np.array([[0, 1, 1], [2, 2, 1], [0, 2, 0]])  # 45 degree
    k90 = np.array([[1, 1, 1], [2, 2, 2], [0, 0, 0]])  # 90 degree
    k135 = np.array([[1, 1, 0], [1, 2, 2], [0, 2, 0]])  # 135 degree
    k180 = np.array([[1, 2, 0], [1, 2, 0], [1, 2, 0]])  # 180 degree
    k225 = np.array([[0, 2, 0], [1, 2, 2], [1, 1, 0]])  # 225 degree
    k270 = np.array([[0, 0, 0], [2, 2, 2], [1, 1, 1]])  # 270 degree
    k315 = np.array([[0, 2, 0], [2, 2, 1], [0, 1, 1]])  # 315 degree
    w = np.zeros((3, 3), dtype=np.uint8)

    # calculating corresponding weights
    w[1, 2] = np.sum(i0 * k0) * (1 / 9)  # w0
    w[0, 2] = np.sum(i45 * k45) * (1 / 9)  # w1
    w[0, 1] = np.sum(i90 * k90) * (1 / 9)  # w2
    w[0, 0] = np.sum(i135 * k135) * (1 / 9)  # w3
    w[1, 0] = np.sum(i180 * k180) * (1 / 9)  # w4
    w[2, 0] = np.sum(i225 * k225) * (1 / 9)  # w5
    w[2, 1] = np.sum(i270 * k270) * (1 / 9)  # w6
    w[2, 2] = np.sum(i315 * k315) * (1 / 9)  # w7
    w[1, 1] = img[2, 2]

    return w  # Returning a 3x3 window of weights calculated


def myfcn_CSNWALBP(g):
    sz = g.shape
    center = (sz[0] // 2, sz[1] // 2)
    j = 0  # threshold value
    result = np.zeros(4, dtype=int)

    for j in np.arange(0, 5, 0.1):
        if g[1, 2] >= g[1, 0] + j:
            result[3] = 1
        if g[0, 2] >= g[2, 0] + j:
            result[2] = 1
        if g[0, 1] >= g[2, 1] + j:
            result[1] = 1
        if g[0, 0] >= g[2, 2] + j:
            result[0] = 1

    sum_val = 0
    szele = result.size
    for j in range(szele):
        sum_val += result[j] * 2**(szele - j - 1)

    return sum_val



start_time = time.time()

# Load an example image (replace with your own image)
original_image = cv2.imread('lena_std.tif', cv2.IMREAD_GRAYSCALE)
original_image = cv2.resize(original_image, (511, 511))


# Specify the amount of padding on all sides
top_pad = 2
bottom_pad = 2
left_pad = 2
right_pad = 2

# Define the value for padding (e.g., 0 for black, 255 for white)
padding_value = 0

# Use the np.pad function to add padding
padded_image = np.pad(original_image, ((top_pad, bottom_pad), (left_pad, right_pad)), mode='constant', constant_values=padding_value)

# Display the padded image
#cv2.imshow('Padded Image', padded_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# NWALBP calculation
sz = padded_image.shape
csnwalbp_image = np.zeros_like(original_image, dtype=int)

# Initialize a cell array to store the 3x3 windows
# Loop through the image and extract 3x3 windows
for i in range(2, sz[0] - 2):
    for j in range(2, sz[1] - 2):
        # Extract the 5 * 5 window centered at (i, j)
        window = padded_image[i-2:i+3, j-2:j+3]
        # Store the window in the cell array
        g = nwlbp_calculation(window)
        sum_val = myfcn_CSNWALBP(g)
        csnwalbp_image[i-2, j-2] = int(sum_val)*255/15

# Display the CSNWALBP image
cv2.imshow('CSNWALBP Image', csnwalbp_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

end_time = time.time()
time_taken_csnwalbp = end_time - start_time

print(time_taken_csnwalbp)

117.4127266407013


In [5]:
print("Time taken for nwalbp:",time_taken_nwalbp)
print("Time taken for csnwalbp:",time_taken_csnwalbp)

Time taken for nwalbp: 201.45676970481873
Time taken for csnwalbp: 117.4127266407013
